<a href="https://colab.research.google.com/github/Hugo99bas/Anal-tica-de-datos/blob/main/Practica2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminares


In [ ]:
import warnings
action = "ignore" # Specifies that warnings will be ignored
warnings.simplefilter(action=action)

In [ ]:
random_state = 27919
import matplotlib
import warnings
action = "ignore" # Specifies that warnings will be ignored
from pathlib import Path
import pandas as pd

path = Path("..") / "/content/parkinsons_disease_data.csv"
index_col = "PatientID"
target = "Diagnosis"
dtype = {target}
data = pd.read_csv(path, index_col= index_col)
#n=5
#data.head(n)
#data.sample(n)

# Preprocesamiento

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:

columns_to_drop = ['DoctorInCharge']
data_cleaned = data.drop(columns=columns_to_drop, axis=1)

# Buscar valores perdidos
missing_values = data_cleaned.isnull().sum()

X = data_cleaned.drop(columns=['Diagnosis'])
y = data_cleaned['Diagnosis']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=random_state)

# Dividir el conjunto inicial de test para hacer un test final con datos desconocidos para los modelos
X_test, X_test_final, y_test, y_test_final = train_test_split(X_scaled, y, test_size=0.5, random_state=random_state)

# Mostrar los tamaños de los conjuntos
print("Tamaño del conjunto de entrenamiento y validación X:", X_train.shape)
print("Tamaño del conjunto de test X:", X_test.shape)
print("Tamaño del conjunto de test final X:", X_test_final.shape)

# Display basic stats about the prepared dataset
X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()


Tamaño del conjunto de entrenamiento y validación X: (1263, 32)
Tamaño del conjunto de test X: (1052, 32)
Tamaño del conjunto de test final X: (1053, 32)


((1263, 32),
 (1052, 32),
 Diagnosis
 1    776
 0    487
 Name: count, dtype: int64,
 Diagnosis
 1    647
 0    405
 Name: count, dtype: int64)

## Funcion optimize

In [ ]:

def optimize(estimator, grid, X, y, cv=5, scoring=None, refit=True):
    """
    Optimiza los hiperparámetros de un modelo usando GridSearchCV con validación cruzada.

    Parámetros:
    - estimator: El modelo a optimizar.
    - grid: Un diccionario con los hiperparámetros a probar.
    - X: Conjunto de características para el entrenamiento.
    - y: Variable objetivo.
    - cv: Número de pliegues para la validación cruzada (default=5).
    - scoring: Métrica(s) para evaluar el modelo (default=None para la métrica predeterminada del modelo).
    - refit: Métrica para ajustar el modelo final (default=True, usa el mejor modelo según la métrica seleccionada).

    Retorna:
    - classifier: El objeto GridSearchCV ajustado con los resultados.
    """
    # Inicializar GridSearchCV
    model = GridSearchCV(
        estimator=estimator,
        param_grid=grid,
        scoring=scoring,  # Métrica utilizada para evaluar los modelos
        n_jobs=-1,  # Usar todos los procesadores disponibles
        refit=refit,  # Ajustar el modelo con la mejor métrica
        cv=cv,  # Número de pliegues para validación cruzada
        return_train_score=True  # Incluir puntuaciones en el conjunto de entrenamiento
    )

    # Ajustar el modelo con los datos
    classifier = model.fit(X, y)

    # Obtener los resultados de la búsqueda
    results = classifier.cv_results_
    results_df = pd.DataFrame(results)

    # Eliminar columnas que contienen resultados individuales de los pliegues
    mask = results_df.columns.str.startswith("split")
    results_df = results_df.loc[:, ~mask]

    #print("Columnas de results_df:", results_df.columns)

    # Ordenar los resultados según la métrica seleccionada
    #sort_by = (by='rank_test_score')  # Ordenar por la métrica o el puntaje predeterminado
    results_df = results_df.sort_values(by='rank_test_score')

    # Mostrar los resultados como una tabla
    display(results_df)

    return classifier


## Funcion evaluate

In [ ]:
def evaluate(estimators, metrics, X, y):
    """Evaluate the estimators according to the metrics."""
    # Create a dataframe to store the results
    index = list(estimators)
    columns = list(metrics)
    results = pd.DataFrame(index = index, columns = columns)

    for metric in metrics:
        for name in estimators:
            # Get the estimator corresponding to the name
            estimator = estimators[name]

            # Get the scorer corresponding to the metric
            scoring = metrics[metric]
            scorer = get_scorer(scoring)

            results.loc[name, metric] = scorer(estimator, X, y)

    display(results)

# Vecinos mas cercanos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import get_scorer_names
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import get_scorer

## Modelo Basico

In [ ]:
# Modelo basico
knn = KNeighborsClassifier(n_neighbors=5)

# Entrenamiento
knn.fit(X_train, y_train)

# PRedicciones
y_pred_knn = knn.predict(X_test)
y_prob_knn = knn.predict_proba(X_test)[:, 1]

# Evaluacion del modelo
accuracy_train_knn = knn.score(X_train, y_train)
accuracy_test_knn = accuracy_score(y_test, y_pred_knn)
roc_auc_knn = roc_auc_score(y_test, y_prob_knn)
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)
classification_report_knn = classification_report(y_test, y_pred_knn)

print("KNN - Modelo básico:")
print("Accuracy Train", accuracy_train_knn)
print("Accuracy Test", accuracy_test_knn)
print("Precision:", precision_knn)
print("Roc-Auc:", roc_auc_knn)
print("Recall:", recall_knn)
print("F1 Score:", f1_knn)

print("Reporte de clasificación:\n", classification_report_knn)



KNN - Modelo básico:
Accuracy Train 0.8226444972288203
Accuracy Test 0.8298479087452472
Precision: 0.8524096385542169
Roc-Auc: 0.9013299750033392
Recall: 0.874806800618238
F1 Score: 0.8634630053394355
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.79      0.76      0.77       405
           1       0.85      0.87      0.86       647

    accuracy                           0.83      1052
   macro avg       0.82      0.82      0.82      1052
weighted avg       0.83      0.83      0.83      1052



## Optimización y entrenamiento

In [ ]:

#Optimizacion de hiperparametros
grid_knn = {
    'n_neighbors': range(1, 30),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
    'algorithm': ['auto', 'ball_tree','kd_tree','brute']
}

knn_op = optimize(
    estimator=knn,  # Modelo base
    grid=grid_knn,  # Hiperparámetros a probar
    X=X_train,  # Conjunto de características de entrenamiento
    y=y_train,  # Etiquetas de entrenamiento
    cv=5,  # Validación cruzada con 5 pliegues
    scoring='accuracy',
    refit='accuracy'
)

# Mejores Hiperparametros
print("Mejores hiperparametros: ", knn_op.best_params_)

# Entrenamiento
knn_op.fit(X_train, y_train)

# PRedicciones
y_pred_knn_op = knn_op.predict(X_test)
y_prob_knn_op = knn.predict_proba(X_test)[:, 1]


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_weights,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
457,0.000927,0.000046,0.011671,0.000522,brute,manhattan,26,distance,"{'algorithm': 'brute', 'metric': 'manhattan', ...",0.773584,0.030374,1,1.000000,0.000000
341,0.003410,0.000133,0.018293,0.000396,kd_tree,manhattan,26,distance,"{'algorithm': 'kd_tree', 'metric': 'manhattan'...",0.773584,0.030374,1,1.000000,0.000000
109,0.000962,0.000112,0.011425,0.000753,auto,manhattan,26,distance,"{'algorithm': 'auto', 'metric': 'manhattan', '...",0.773584,0.030374,1,1.000000,0.000000
225,0.002601,0.000168,0.013226,0.000864,ball_tree,manhattan,26,distance,"{'algorithm': 'ball_tree', 'metric': 'manhatta...",0.773584,0.030374,1,1.000000,0.000000
231,0.002693,0.000092,0.013795,0.000481,ball_tree,manhattan,29,distance,"{'algorithm': 'ball_tree', 'metric': 'manhatta...",0.772793,0.030686,5,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0.003246,0.000124,0.026535,0.001371,kd_tree,euclidean,2,uniform,"{'algorithm': 'kd_tree', 'metric': 'euclidean'...",0.621595,0.037601,457,0.830565,0.010636
176,0.002847,0.000152,0.022368,0.001179,ball_tree,manhattan,2,uniform,"{'algorithm': 'ball_tree', 'metric': 'manhatta...",0.616808,0.029345,461,0.822644,0.004768
408,0.001260,0.000118,0.010676,0.000708,brute,manhattan,2,uniform,"{'algorithm': 'brute', 'metric': 'manhattan', ...",0.616808,0.029345,461,0.822644,0.004768
292,0.003482,0.000317,0.027819,0.001674,kd_tree,manhattan,2,uniform,"{'algorithm': 'kd_tree', 'metric': 'manhattan'...",0.616808,0.029345,461,0.822644,0.004768


Mejores hiperparametros:  {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 26, 'weights': 'distance'}


## Evaluación

In [ ]:

# Evaluacion del modelo
accuracy_train_knn_op = knn_op.score(X_train, y_train)
accuracy_test_knn_op = accuracy_score(y_test, y_pred_knn_op)
roc_auc_knn_op = roc_auc_score(y_test, y_prob_knn_op)
precision_knn_op = precision_score(y_test, y_pred_knn_op)
recall_knn_op = recall_score(y_test, y_pred_knn_op)
f1_knn_op = f1_score(y_test, y_pred_knn_op)
classification_report_knn_op = classification_report(y_test, y_pred_knn_op)

print("KNN - Modelo optimizado")
print("Accuracy Train", accuracy_train_knn_op)
print("Accuracy Test", accuracy_test_knn_op)
print("Precision:", precision_knn_op)
print("Roc-Auc:", roc_auc_knn_op)
print("Recall:", recall_knn_op)
print("F1 Score:", f1_knn_op)

print("Reporte de clasificación:\n", classification_report_knn_op)



KNN - Modelo optimizado
Accuracy Train 1.0
Accuracy Test 1.0
Precision: 1.0
Roc-Auc: 0.9013299750033392
Recall: 1.0
F1 Score: 1.0
Reporte de clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       405
           1       1.00      1.00      1.00       647

    accuracy                           1.00      1052
   macro avg       1.00      1.00      1.00      1052
weighted avg       1.00      1.00      1.00      1052



## Test final

In [ ]:
# Predicciones
y_pred_final_knn_op = knn_op.predict(X_test_final)
y_prob_final_knn_op = knn_op.predict_proba(X_test_final)[:, 1]

# Evaluacion del modelo
accuracy_test_final_knn_op = accuracy_score(y_test_final, y_pred_final_knn_op)
roc_auc_final_knn_op = roc_auc_score(y_test_final, y_prob_final_knn_op)
classification_report_final_knn_op = classification_report(y_test_final, y_pred_final_knn_op)
precision_final_knn_op = precision_score(y_test_final, y_pred_final_knn_op)
recall_final_knn_op = recall_score(y_test_final, y_pred_final_knn_op)
f1_final_knn_op = f1_score(y_test_final, y_pred_final_knn_op)

# Metricas
print("KNN - Modelo optimizado Test Final")
print("Accuracy", accuracy_test_final_knn_op)
print("Precision:", precision_final_knn_op)
print("Roc-Auc:", roc_auc_final_knn_op)
print("Recall:", recall_final_knn_op)
print("F1 Score:", f1_final_knn_op)

print("Reporte de clasificación:\n", classification_report_final_knn_op)

KNN - Modelo optimizado Test Final
Accuracy 0.8347578347578347
Precision: 0.8484848484848485
Roc-Auc: 0.9127961502390726
Recall: 0.8949771689497716
F1 Score: 0.8711111111111111
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.73      0.77       396
           1       0.85      0.89      0.87       657

    accuracy                           0.83      1053
   macro avg       0.83      0.81      0.82      1053
weighted avg       0.83      0.83      0.83      1053



## Comparacion modelos

In [ ]:
#print(get_scorer_names())
estimators = {"Knn basico": knn, "Knn optimizado": knn_op}
metrics = {"Accuracy": "accuracy", "Precision":"precision", "Roc-Auc":"roc_auc", "Recall":"recall","F1":"f1"}

print("Evaluacion")
evaluate(estimators, metrics, X_test, y_test)


print("Evaluacion Test final")
evaluate(estimators, metrics, X_test_final, y_test_final)

Evaluacion


,Accuracy,Precision,Roc-Auc,Recall,F1
Knn basico,0.829848,0.85241,0.90133,0.874807,0.863463
Knn optimizado,1.0,1.0,1.0,1.0,1.0


Evaluacion Test final


,Accuracy,Precision,Roc-Auc,Recall,F1
Knn basico,0.725546,0.77381,0.774764,0.791476,0.782543
Knn optimizado,0.834758,0.848485,0.912796,0.894977,0.871111


# Adaptative Boosting

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

## Modelo Básico

In [ ]:
# Modelo basico
tree_base = DecisionTreeClassifier(max_depth=1,random_state= random_state)
ada_boost = AdaBoostClassifier(
    tree_base,
    n_estimators=50,           # Número de clasificadores débiles
    learning_rate=1.0,         # Tasa de aprendizaje
    random_state=random_state         # Semilla para reproducibilidad
)

# Entrenamiento
ada_boost.fit(X_train, y_train)

# Predicciones
y_pred_ada_boost = ada_boost.predict(X_test)
y_prob_ada_boost = ada_boost.predict_proba(X_test)[:, 1]

# Evaluacion del modelo
accuracy_train_ada_boost = ada_boost.score(X_train, y_train)
accuracy_test_ada_boost = accuracy_score(y_test, y_pred_ada_boost)
roc_auc_ada_boost = roc_auc_score(y_test, y_prob_ada_boost)
precision_ada_boost = precision_score(y_test, y_pred_ada_boost)
recall_ada_boost = recall_score(y_test, y_pred_ada_boost)
f1_ada_boost = f1_score(y_test, y_pred_ada_boost)
classification_report_ada_boost = classification_report(y_test, y_pred_ada_boost)


# Metricas
print("Adaptative Boosting - Modelo básico (Con tree decision)")
print("Accuracy Train", accuracy_train_ada_boost)
print("Accuracy Test", accuracy_test_ada_boost)
print("Precision:", precision_ada_boost)
print("Roc-Auc:", roc_auc_ada_boost)
print("Recall:", recall_ada_boost)
print("F1 Score:", f1_ada_boost)

print("Reporte de clasificación:\n", classification_report_ada_boost)

Adaptative Boosting - Modelo básico (Con tree decision)
Accuracy Train 0.9334916864608076
Accuracy Test 0.9315589353612167
Precision: 0.9336349924585219
Roc-Auc: 0.9776060449939894
Recall: 0.9567233384853169
F1 Score: 0.9450381679389313
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.93      0.89      0.91       405
           1       0.93      0.96      0.95       647

    accuracy                           0.93      1052
   macro avg       0.93      0.92      0.93      1052
weighted avg       0.93      0.93      0.93      1052



## Optimizacion y entrenamiento

In [ ]:

# Definir los hiperparámetros a probar
grid_ada_boost = {
    'n_estimators': [10, 50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0],
    'algorithm': ['SAMME', 'SAMME.R']
}

ada_boost_op = optimize(
    estimator = ada_boost,
    grid=grid_ada_boost,
    X=X_train,
    y=y_train,
    cv=5,
    scoring='accuracy',
    refit='accuracy'
)

print("Mejores parametros para Adaptative Boosting: ", ada_boost_op.best_params_)

# Ajustar el modelo
ada_boost_op.fit(X_train, y_train)

# Predicciones
y_pred_ada_boost_op = ada_boost_op.predict(X_test)
y_prob_ada_boost_op = ada_boost_op.predict_proba(X_test)[:, 1]




,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_learning_rate,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
23,0.727183,0.007862,0.032975,0.001600,SAMME.R,0.10,200,"{'algorithm': 'SAMME.R', 'learning_rate': 0.1,...",0.923998,0.009431,1,0.935670,0.006216
10,0.347621,0.005873,0.013634,0.000130,SAMME,0.50,100,"{'algorithm': 'SAMME', 'learning_rate': 0.5, '...",0.921629,0.013938,2,0.936461,0.003146
22,0.366040,0.009483,0.018960,0.002434,SAMME.R,0.10,100,"{'algorithm': 'SAMME.R', 'learning_rate': 0.1,...",0.920804,0.014907,3,0.928346,0.011619
25,0.188184,0.005115,0.009884,0.000842,SAMME.R,0.50,50,"{'algorithm': 'SAMME.R', 'learning_rate': 0.5,...",0.919258,0.013742,4,0.937846,0.004218
11,0.733515,0.016503,0.029785,0.002738,SAMME,0.50,200,"{'algorithm': 'SAMME', 'learning_rate': 0.5, '...",0.918477,0.020313,5,0.941015,0.005879
7,0.699078,0.016163,0.027659,0.001862,SAMME,0.10,200,"{'algorithm': 'SAMME', 'learning_rate': 0.1, '...",0.917658,0.015702,6,0.929730,0.010500
9,0.173225,0.003714,0.007514,0.000268,SAMME,0.50,50,"{'algorithm': 'SAMME', 'learning_rate': 0.5, '...",0.917658,0.014448,6,0.931512,0.007857
13,0.186648,0.011475,0.008361,0.000724,SAMME,1.00,50,"{'algorithm': 'SAMME', 'learning_rate': 1.0, '...",0.916099,0.022197,8,0.939036,0.003958
14,0.391611,0.016015,0.015839,0.001184,SAMME,1.00,100,"{'algorithm': 'SAMME', 'learning_rate': 1.0, '...",0.912143,0.017454,9,0.941017,0.008551
26,0.366938,0.008312,0.018406,0.001548,SAMME.R,0.50,100,"{'algorithm': 'SAMME.R', 'learning_rate': 0.5,...",0.908962,0.019468,10,0.947744,0.005167


Mejores parametros para Adaptative Boosting:  {'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 200}


## Evaluacion

In [ ]:
# Evaluacion del modelo optimizado
accuracy_train_ada_boost_op = ada_boost_op.score(X_train, y_train)
accuracy_test_ada_boost_op = accuracy_score(y_test, y_pred_ada_boost_op)
roc_auc_ada_boost_op = roc_auc_score(y_test, y_prob_ada_boost_op)
precision_ada_boost_op = precision_score(y_test, y_pred_ada_boost_op)
recall_ada_boost_op = recall_score(y_test, y_pred_ada_boost_op)
f1_ada_boost_op = f1_score(y_test, y_pred_ada_boost_op)
classification_report_ada_boost_op = classification_report(y_test, y_pred_ada_boost_op)

# Metricas
print("Adaptative Boosting - Modelo optimizado")
print("Accuracy Train", accuracy_train_ada_boost_op)
print("Accuracy Test", accuracy_test_ada_boost_op)
print("Precision:", precision_ada_boost_op)
print("Roc-Auc:", roc_auc_ada_boost_op)
print("Recall:", recall_ada_boost_op)
print("F1 Score:", f1_ada_boost_op)

print("Reporte de clasificación:\n", classification_report_ada_boost_op)


Adaptative Boosting - Modelo optimizado
Accuracy Train 0.9334916864608076
Accuracy Test 0.935361216730038
Precision: 0.9276218611521418
Roc-Auc: 0.9709638025454613
Recall: 0.9706336939721792
F1 Score: 0.9486404833836858
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.95      0.88      0.91       405
           1       0.93      0.97      0.95       647

    accuracy                           0.94      1052
   macro avg       0.94      0.92      0.93      1052
weighted avg       0.94      0.94      0.93      1052



## Test Final

In [ ]:
# Predicciones
y_pred_final_ada_boost_op = ada_boost_op.predict(X_test_final)
y_prob_final_ada_boost_op = ada_boost_op.predict_proba(X_test_final)[:, 1]

# Evaluacion del modelo
accuracy_test_final_ada_boost_op = accuracy_score(y_test_final, y_pred_final_ada_boost_op)
roc_auc_final_ada_boost_op = roc_auc_score(y_test_final, y_prob_final_ada_boost_op)
precision_final_ada_boost_op = precision_score(y_test_final, y_pred_final_ada_boost_op)
recall_final_ada_boost_op = recall_score(y_test_final, y_pred_final_ada_boost_op)
f1_final_ada_boost_op = f1_score(y_test_final, y_pred_final_ada_boost_op)

classification_report_final_ada_boost_op = classification_report(y_test_final, y_pred_final_ada_boost_op)

# Metricas
print("Adaptative Boosting - Modelo Optimizado Test Final")
print("Accuracy", accuracy_test_final_ada_boost_op)
print("Precision:", precision_final_ada_boost_op)
print("Roc-Auc:", roc_auc_final_ada_boost_op)
print("Recall:", recall_final_ada_boost_op)
print("F1 Score:", f1_final_ada_boost_op)

print("Reporte de clasificación:\n", classification_report_final_ada_boost_op)

Adaptative Boosting - Modelo Optimizado Test Final
Accuracy 0.9240265906932573
Precision: 0.9211678832116789
Roc-Auc: 0.966391464108359
Recall: 0.9604261796042618
F1 Score: 0.940387481371088
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.93      0.86      0.90       396
           1       0.92      0.96      0.94       657

    accuracy                           0.92      1053
   macro avg       0.93      0.91      0.92      1053
weighted avg       0.92      0.92      0.92      1053



## Comparacion modelos

In [ ]:
estimators = {"Ada_boosting": ada_boost, "Ada_boosting optimizado": ada_boost_op}
metrics = {"Accuracy": "accuracy", "Precision":"precision", "Roc-Auc":"roc_auc", "Recall":"recall","F1":"f1"}

print("Evaluacion")
evaluate(estimators, metrics, X_test, y_test)


print("Evaluacion Test final")
evaluate(estimators, metrics, X_test_final, y_test_final)

Evaluacion


,Accuracy,Precision,Roc-Auc,Recall,F1
Ada_boosting,0.931559,0.933635,0.977606,0.956723,0.945038
Ada_boosting optimizado,0.935361,0.927622,0.970964,0.970634,0.94864


Evaluacion Test final


,Accuracy,Precision,Roc-Auc,Recall,F1
Ada_boosting,0.924976,0.933934,0.960649,0.946728,0.940287
Ada_boosting optimizado,0.924027,0.921168,0.966391,0.960426,0.940387


# Bootstrap Aggregating

## Con arbol de decisión

### Modelo basico

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:

bag = BaggingClassifier(
    tree_base,   # Clasificador base
    n_estimators=50,           # Número de clasificadores
    max_samples=0.8,           # Porcentaje de muestras para cada clasificador
    max_features=1.0,          # Porcentaje de características para cada clasificador
    bootstrap=True,            # Muestreo con reemplazo
    ##bootstrap_features=False,  # No hacer muestreo en características
    random_state=random_state           # Semilla para reproducibilidad
)

# Entrenamiento
bag.fit(X_train, y_train)

# Predicciones
y_pred_bag = bag.predict(X_test)
y_prob_bag = bag.predict_proba(X_test)[:, 1]

# Evaluacion del modelo
accuracy_train_bag = bag.score(X_train, y_train)
accuracy_test_bag = accuracy_score(y_test, y_pred_bag)
roc_auc_bag = roc_auc_score(y_test, y_prob_bag)
precision_bag = precision_score(y_test, y_pred_bag)
recall_bag = recall_score(y_test, y_pred_bag)
f1_bag = f1_score(y_test, y_pred_bag)
classification_report_bag = classification_report(y_test, y_pred_bag)


# Metricas
print("Bootstrap Aggregating - Modelo básico")
print("Accuracy Train", accuracy_train_bag)
print("Accuracy Test", accuracy_test_bag)
print("Precision:", precision_bag)
print("Roc-Auc:", roc_auc_bag)
print("Recall:", recall_bag)
print("F1 Score:", f1_bag)

print("Reporte de clasificación:\n", classification_report_bag)


Bootstrap Aggregating - Modelo básico
Accuracy Train 0.7711797307996833
Accuracy Test 0.7709125475285171
Precision: 0.7550251256281407
Roc-Auc: 0.7255767359322228
Recall: 0.9289026275115919
F1 Score: 0.832986832986833
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.82      0.52      0.64       405
           1       0.76      0.93      0.83       647

    accuracy                           0.77      1052
   macro avg       0.79      0.72      0.73      1052
weighted avg       0.78      0.77      0.76      1052



## Optimización y entrenamiento

In [ ]:

# Hiperparámetros
grid_bag = {
    'n_estimators': [10, 50, 100, 200],  # Número de clasificadores
    'max_samples': [0.5, 0.8, 1.0],  # Porcentaje de muestras
    'max_features': [0.5, 0.8, 1.0]  # Porcentaje de características
}

# Ejecutar la optimización
bag_op = optimize(
    estimator=bag,
    grid=grid_bag,
    X=X_train,
    y=y_train,
    cv=5,  # Validación cruzada con 5 pliegues
    scoring='accuracy',  # Métrica para optimizar
    refit='accuracy'
)

print("Mejores parametros para Adaptative Boosting: ", bag_op.best_params_)

# Ajustar el modelo
bag_op.fit(X_train, y_train)

# Predicciones
y_pred_bag_op = bag_op.predict(X_test)
y_prob_bag_op = bag_op.predict_proba(X_test)[:, 1]



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_max_samples,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.024295,0.001378,0.002477,0.000125,0.5,0.5,10,"{'max_features': 0.5, 'max_samples': 0.5, 'n_e...",0.770393,0.006415,1,0.770784,0.001877
4,0.024328,0.001063,0.002462,0.000099,0.5,0.8,10,"{'max_features': 0.5, 'max_samples': 0.8, 'n_e...",0.770393,0.006415,1,0.771180,0.001637
12,0.025642,0.002366,0.002479,0.000161,0.8,0.5,10,"{'max_features': 0.8, 'max_samples': 0.5, 'n_e...",0.770393,0.006415,1,0.770784,0.002342
8,0.025330,0.000573,0.002310,0.000090,0.5,1.0,10,"{'max_features': 0.5, 'max_samples': 1.0, 'n_e...",0.770393,0.006415,1,0.770586,0.001991
25,0.127581,0.003657,0.007524,0.000465,1.0,0.5,50,"{'max_features': 1.0, 'max_samples': 0.5, 'n_e...",0.770393,0.006415,1,0.771180,0.001637
28,0.029405,0.001366,0.002543,0.000187,1.0,0.8,10,"{'max_features': 1.0, 'max_samples': 0.8, 'n_e...",0.770393,0.006415,1,0.771180,0.001637
20,0.029319,0.002348,0.002382,0.000212,0.8,1.0,10,"{'max_features': 0.8, 'max_samples': 1.0, 'n_e...",0.770393,0.006415,1,0.771180,0.001637
21,0.137860,0.004162,0.008009,0.001029,0.8,1.0,50,"{'max_features': 0.8, 'max_samples': 1.0, 'n_e...",0.770393,0.006415,1,0.771180,0.001637
35,0.589401,0.015939,0.025962,0.001018,1.0,1.0,200,"{'max_features': 1.0, 'max_samples': 1.0, 'n_e...",0.770393,0.006415,1,0.771180,0.001637
32,0.031501,0.001277,0.002424,0.000153,1.0,1.0,10,"{'max_features': 1.0, 'max_samples': 1.0, 'n_e...",0.770393,0.006415,1,0.771180,0.001637


Mejores parametros para Adaptative Boosting:  {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 10}


## Evaluacion

In [ ]:
# Evaluacion del modelo optimizado
accuracy_train_bag_op = bag_op.score(X_train, y_train)
accuracy_test_bag_op = accuracy_score(y_test, y_pred_bag_op)
roc_auc_bag_op = roc_auc_score(y_test, y_prob_bag_op)
precision_bag_op = precision_score(y_test, y_pred_bag_op)
recall_bag_op = recall_score(y_test, y_pred_bag_op)
f1_bag_op = f1_score(y_test, y_pred_bag_op)
classification_report_bag_op = classification_report(y_test, y_pred_bag_op)

# Metricas
print("Bootstrap Aggregating - Modelo optimizado")
print("Accuracy Train", accuracy_train_bag_op)
print("Accuracy Test", accuracy_test_bag_op)
print("Precision:", precision_bag_op)
print("Roc-Auc:", roc_auc_bag_op)
print("Recall:", recall_bag_op)
print("F1 Score:", f1_bag_op)

print("Reporte de clasificación:\n", classification_report_bag_op)

Bootstrap Aggregating - Modelo optimizado
Accuracy Train 0.770387965162312
Accuracy Test 0.7709125475285171
Precision: 0.75375
Roc-Auc: 0.844171580132425
Recall: 0.9319938176197836
F1 Score: 0.8334485141672425
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.83      0.51      0.63       405
           1       0.75      0.93      0.83       647

    accuracy                           0.77      1052
   macro avg       0.79      0.72      0.73      1052
weighted avg       0.78      0.77      0.76      1052



## Con vecinos mas cercanos

### Modelo basico

In [ ]:

# Modelo Bagging con KNN básico definido anteriormente
bag_knn = BaggingClassifier(
    knn,  # Modelo base KNN
    n_estimators=50,          # Número de clasificadores
    max_samples=0.8,          # Porcentaje de muestras
    max_features=1.0,         # Porcentaje de características
    bootstrap=True,           # Muestreo con reemplazo
    random_state=random_state          # Semilla para reproducibilidad
)

# Entrenar el modelo básico
bag_knn.fit(X_train, y_train)

## Predicciones
y_pred_bag_knn = bag_knn.predict(X_test)
y_prob_bag_knn = bag_knn.predict_proba(X_test)[:, 1]

# Evaluacion
accuracy_train_bag_knn = bag_knn.score(X_train, y_train)
accuracy_test_bag_knn = accuracy_score(y_test, y_pred_bag_knn)
roc_auc_bag_knn = roc_auc_score(y_test, y_prob_bag_knn)
precision_bag_knn = precision_score(y_test_final, y_pred_bag_knn)
recall_bag_knn = recall_score(y_test_final, y_pred_bag_knn)
f1_bag_knn = f1_score(y_test_final, y_pred_bag_knn)

classification_report_final_bag_knn = classification_report(y_test, y_pred_bag_knn)

# Metricas
print("Bootstrap Aggregating - Modelo basico")
print("Accuracy Train:", accuracy_train_bag_knn)
print("Accuracy Test:", accuracy_test_bag_knn)
print("Precision:", precision_bag_knn)
print("Roc-Auc:", roc_auc_bag_knn)
print("Recall:", recall_bag_knn)
print("F1 Score:", f1_bag_knn)

print("Reporte de clasificación:\n", classification_report_final_bag_knn)

ValueError: Found input variables with inconsistent numbers of samples: [1053, 1052]

### Optimizacion y entrenamiento

In [ ]:

grid_bag_knn = {
    'estimator__n_neighbors': [5, 10, 20],
    'estimator__weights': ['uniform', 'distance'],
    'n_estimators': [50, 100, 150],
    'max_samples': [0.6, 0.8, 1.0],
    'max_features': [0.6, 0.8, 1.0]
}

bag_knn_op = optimize(
    estimator = bag_knn,
    grid=grid_bag_knn,
    X=X_train,
    y=y_train,
    cv=5,
    scoring='accuracy',
    refit='accuracy'
)

# Entrenar el modelo optimizado
bag_knn_op.fit(X_train, y_train)

# Predicciones
y_pred_bag_knn_op = bag_knn_op.predict(X_test)
y_prob_bag_knn_op = bag_knn_op.predict_proba(X_test)[:, 1]


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__n_neighbors,param_estimator__weights,param_max_features,param_max_samples,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
144,0.058532,0.000975,0.132610,0.006660,20,distance,0.8,0.6,50,"{'estimator__n_neighbors': 20, 'estimator__wei...",0.763301,0.027932,1,0.999604,0.000485
117,0.059624,0.001705,0.131306,0.002214,20,uniform,0.8,0.6,50,"{'estimator__n_neighbors': 20, 'estimator__wei...",0.762504,0.025822,2,0.787417,0.013691
35,0.182076,0.002259,0.297495,0.008430,5,distance,0.6,1.0,150,"{'estimator__n_neighbors': 5, 'estimator__weig...",0.762488,0.018489,3,1.000000,0.000000
125,0.277559,0.051897,0.514013,0.048949,20,uniform,0.8,1.0,150,"{'estimator__n_neighbors': 20, 'estimator__wei...",0.761723,0.029199,4,0.811563,0.007842
8,0.189889,0.004283,0.286843,0.009662,5,uniform,0.6,1.0,150,"{'estimator__n_neighbors': 5, 'estimator__weig...",0.761698,0.019366,5,0.935078,0.007236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,0.059650,0.000981,0.104193,0.006009,5,uniform,1.0,1.0,50,"{'estimator__n_neighbors': 5, 'estimator__weig...",0.707115,0.037874,158,0.843230,0.006644
25,0.121434,0.001902,0.195609,0.004989,5,uniform,1.0,1.0,100,"{'estimator__n_neighbors': 5, 'estimator__weig...",0.702350,0.035432,159,0.845014,0.005454
52,0.139057,0.007061,0.244811,0.007959,5,distance,1.0,1.0,100,"{'estimator__n_neighbors': 5, 'estimator__weig...",0.699975,0.035946,160,1.000000,0.000000
26,0.186761,0.002184,0.306699,0.004642,5,uniform,1.0,1.0,150,"{'estimator__n_neighbors': 5, 'estimator__weig...",0.698397,0.037361,161,0.844222,0.004726


## Evaluacion

In [ ]:

# Evaluacion
accuracy_test_bag_knn_op = accuracy_score(y_test, y_pred_bag_knn_op)
roc_auc_bag_knn_op = roc_auc_score(y_test, y_prob_bag_knn_op)
precision_bag_knn_op = precision_score(y_test, y_pred_bag_knn_op)
recall_bag_knn_op = recall_score(y_test, y_pred_bag_knn_op)
f1_bag_knn_op = f1_score(y_test, y_pred_bag_knn_op)
classification_report_bag_knn_op = classification_report(y_test, y_pred_bag_knn_op)


# Metricas
print("Bootstrap aggregating - Modelo optimizado")
print("Accuracy", accuracy_test_bag_knn_op)
print("Precision:", precision_bag_knn_op)
print("Roc-Auc:", roc_auc_bag_knn_op)
print("Recall:", recall_bag_knn_op)
print("F1 Score:", f1_bag_knn_op)

print("Reporte de clasificación:\n", classification_report_bag_knn_op)

Bootstrap aggregating - Modelo optimizado
Accuracy 1.0
Precision: 1.0
Roc-Auc: 1.0
Recall: 1.0
F1 Score: 1.0
Reporte de clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       405
           1       1.00      1.00      1.00       647

    accuracy                           1.00      1052
   macro avg       1.00      1.00      1.00      1052
weighted avg       1.00      1.00      1.00      1052



## Test Final

In [ ]:
# Predicciones
y_pred_final_bag_knn_op = bag_knn_op.predict(X_test_final)
y_prob_final_bag_knn_op = bag_knn_op.predict_proba(X_test_final)[:, 1]

# Evaluacion del modelo
accuracy_test_final_bag_knn_op = accuracy_score(y_test_final, y_pred_final_bag_knn_op)
roc_auc_final_bag_knn_op = roc_auc_score(y_test_final, y_prob_final_bag_knn_op)
classification_report_final_bag_knn_op = classification_report(y_test_final, y_pred_final_bag_knn_op)
precision_final_bag_knn_op = precision_score(y_test_final, y_pred_final_bag_knn_op)
recall_final_bag_knn_op = recall_score(y_test_final, y_pred_final_bag_knn_op)
f1_final_bag_knn_op = f1_score(y_test_final, y_pred_final_bag_knn_op)

# Metricas
print("KNN - Modelo optimizado Test Final")
print("Accuracy", accuracy_test_final_bag_knn_op)
print("Precision:", precision_final_bag_knn_op)
print("Roc-Auc:", roc_auc_final_bag_knn_op)
print("Recall:", recall_final_bag_knn_op)
print("F1 Score:", f1_final_bag_knn_op)

print("Reporte de clasificación:\n", classification_report_final_bag_knn_op)

KNN - Modelo optimizado Test Final
Accuracy 0.8214624881291548
Precision: 0.8041504539559015
Roc-Auc: 0.919395630582845
Recall: 0.943683409436834
F1 Score: 0.8683473389355743
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.87      0.62      0.72       396
           1       0.80      0.94      0.87       657

    accuracy                           0.82      1053
   macro avg       0.84      0.78      0.80      1053
weighted avg       0.83      0.82      0.81      1053



## Comparación modelos

In [ ]:

estimators = {"Bag basico": bag, "Bag optimizado": bag_op, "Bag_Knn basico": bag_knn, "Bag_Knn_Optimizado": bag_knn_op}
metrics = {"Accuracy": "accuracy", "Precision":"precision", "Roc-Auc":"roc_auc", "Recall":"recall","F1":"f1"}

print("Evaluacion")
evaluate(estimators, metrics, X_test, y_test)


print("Evaluacion Test final")
evaluate(estimators, metrics, X_test_final, y_test_final)

Evaluacion


,Accuracy,Precision,Roc-Auc,Recall,F1
Bag basico,0.770913,0.755025,0.725577,0.928903,0.832987
Bag optimizado,0.770913,0.75375,0.844172,0.931994,0.833449
Bag_Knn basico,0.843156,0.849275,0.921261,0.905719,0.876589
Bag_Knn_Optimizado,1.0,1.0,1.0,1.0,1.0


Evaluacion Test final


,Accuracy,Precision,Roc-Auc,Recall,F1
Bag basico,0.778727,0.761728,0.726621,0.939117,0.841172
Bag optimizado,0.776828,0.758578,0.845114,0.942161,0.840462
Bag_Knn basico,0.737892,0.775687,0.807468,0.81583,0.795252
Bag_Knn_Optimizado,0.821462,0.80415,0.919396,0.943683,0.868347


# Random Forest

## Modelo basico

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [ ]:

rf = RandomForestClassifier(
    n_estimators=100,       # Número de árboles
    max_depth=None,         # Sin límite en la profundidad de los árboles
    max_features='sqrt',    # Raíz cuadrada del número de características
    bootstrap=True,         # Muestreo con reemplazo
    random_state=42         # Semilla para reproducibilidad
)

# Entrenamiento
rf.fit(X_train, y_train)

# Evaluar el modelo
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

# Evaluacion del modelo
accuracy_train_rf = bag.score(X_train, y_train)
accuracy_test_rf = accuracy_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_prob_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
classification_report_rf = classification_report(y_test, y_pred_rf)


# Metricas
print("Random Forest - Modelo básico")
print("Accuracy Train", accuracy_train_rf)
print("Accuracy Test", accuracy_test_rf)
print("Precision:", precision_rf)
print("Roc-Auc:", roc_auc_rf)
print("Recall:", recall_rf)
print("F1 Score:", f1_rf)

print("Reporte de clasificación:\n", classification_report_rf)


## Optimizacion y entrenamiento

In [ ]:

grid_rf = {
    'n_estimators': [3, 5, 10, 20, 50, 100, 200],      # Número de árboles
    'criterion': ['gini', 'entropy'],                  # Criterio de división
    'max_depth': [None, 10, 20, 30],    # Profundidad máxima de los árboles
    'max_features': ['sqrt', 'log2'],   # Número máximo de características
    'min_samples_split': [2, 5, 10],    # Muestras mínimas para dividir un nodo
    'min_samples_leaf': [1, 2, 4]       # Muestras mínimas en una hoja
}

rf_op = optimize(
    estimator=rf,  # Modelo base
    grid=grid_rf,  # Hiperparámetros a probar
    X=X_train,  # Conjunto de características de entrenamiento
    y=y_train,  # Etiquetas de entrenamiento
    cv=5,  # Validación cruzada con 5 pliegues
    scoring='accuracy',
    refit='accuracy'
)

# Entrenamiento
rf_op.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", rf_op.best_params_)

# Predicciones
y_pred_rf_op = rf_op.predict(X_test)
y_prob_rf_op = rf_op.predict_proba(X_test)[:, 1]


## Evaluación

In [ ]:

# Evaluacion del modelo
accuracy_test_rf_op = accuracy_score(y_test, y_pred_rf_op)
roc_auc_rf_op = roc_auc_score(y_test, y_prob_rf_op)
precision_rf_op = precision_score(y_test, y_pred_rf_op)
recall_rf_op = recall_score(y_test, y_pred_rf_op)
f1_rf_op = f1_score(y_test, y_pred_rf_op)

classification_report_rf_op = classification_report(y_test, y_pred_rf_op)


# Metricas
print("Random Forest - Modelo optimizado")
print("Accuracy", accuracy_test_rf_op)
print("Precision:", precision_rf_op)
print("Roc-Auc:", roc_auc_rf_op)
print("Recall:", recall_rf_op)
print("F1 Score:", f1_rf_op)

print("Reporte de clasificación:\n", classification_report_rf_op)

## Test Final

In [ ]:
# Predicciones
y_pred_final_rf_op = rf_op.predict(X_test_final)
y_prob_final_rf_op = rf_op.predict_proba(X_test_final)[:, 1]

# Evaluacion del modelo
accuracy_test_final_rf_op = accuracy_score(y_test_final, y_pred_final_rf_op)
roc_auc_final_rf_op = roc_auc_score(y_test_final, y_prob_final_rf_op)
classification_report_final_rf_op = classification_report(y_test_final, y_pred_final_rf_op)
precision_final_rf_op = precision_score(y_test_final, y_pred_final_rf_op)
recall_final_rf_op = recall_score(y_test_final, y_pred_final_rf_op)
f1_final_rf_op = f1_score(y_test_final, y_pred_final_rf_op)

# Metricas
print("KNN - Modelo optimizado Test Final")
print("Accuracy", accuracy_test_final_rf_op)
print("Precision:", precision_final_rf_op)
print("Roc-Auc:", roc_auc_final_rf_op)
print("Recall:", recall_final_rf_op)
print("F1 Score:", f1_final_rf_op)

print("Reporte de clasificación:\n", classification_report_final_rf_op)

## Comparación modelos

In [ ]:

estimators = {"Random forest basico": rf, "Random forest optimizado": rf_op}
metrics = {"Accuracy": "accuracy", "Precision":"precision", "Roc-Auc":"roc_auc", "Recall":"recall","F1":"f1"}

print("Evaluacion")
evaluate(estimators, metrics, X_test, y_test)


print("Evaluacion Test final")
evaluate(estimators, metrics, X_test_final, y_test_final)

# Gradient Boosting

## Modelo basico

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:

# Modelo
gb = GradientBoostingClassifier(
    n_estimators=100,         # Número de árboles
    learning_rate=0.1,        # Tasa de aprendizaje
    max_depth=3,              # Profundidad máxima de los árboles
    random_state=42           # Semilla para reproducibilidad
)

# Entrenamiento
gb.fit(X_train, y_train)

# Predicciones
y_pred_gb = gb.predict(X_test)
y_prob_gb = gb.predict_proba(X_test)[:, 1]

# Evaluacion del modelo
accuracy_train_gb = gb.score(X_train, y_train)
accuracy_test_gb = accuracy_score(y_test, y_pred_gb)
roc_auc_gb = roc_auc_score(y_test, y_prob_gb)
precision_gb = precision_score(y_test, y_pred_gb)
recall_gb = recall_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb)
classification_report_gb = classification_report(y_test, y_pred_gb)


# Metricas
print("Gradient Boosting - Modelo básico")
print("Accuracy Train", accuracy_train_gb)
print("Accuracy Test", accuracy_test_gb)
print("Precision:", precision_gb)
print("Roc-Auc:", roc_auc_gb)
print("Recall:", recall_gb)
print("F1 Score:", f1_gb)

print("Reporte de clasificación:\n", classification_report_gb)

Gradient Boosting - Modelo básico:
Accuracy Train: 0.9703087885985748
Accuracy Test: 0.9073634204275535
ROC AUC: 0.9540098400984011
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.85      0.90      0.87       150
           1       0.94      0.91      0.93       271

    accuracy                           0.91       421
   macro avg       0.90      0.91      0.90       421
weighted avg       0.91      0.91      0.91       421



## Optimización

In [ ]:

grid_gb = {
    'n_estimators': [10, 20, 50, 100, 200],       # Número de árboles
    'learning_rate': [0.01, 0.05, 0.1, 0.2],   # Tasa de aprendizaje
    'loss': ['deviance', 'exponential'], # Función de pérdida
    'criterion': ['friedman_mse', 'squared_error'],
    'max_depth': [1, 3, 5, 7],              # Profundidad máxima
    'min_samples_split': [2, 5, 10],     # Muestras mínimas para dividir un nodo
    'min_samples_leaf': [1, 2, 4]        # Muestras mínimas en una hoja
}

gb_op = optimize(
    estimator=gb,  # Modelo base
    grid=grid_gb,  # Hiperparámetros a probar
    X=X_train,  # Conjunto de características de entrenamiento
    y=y_train,  # Etiquetas de entrenamiento
    cv=5,  # Validación cruzada con 5 pliegues
    scoring='accuracy',
    refit='accuracy'
)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", gb_op.best_params_)

gb_op.fit(X_train, y_train)

# Predicciones
y_pred_gb_op = gb_op.predict(X_test)
y_prob_gb_op = gb_op.predict_proba(X_test)[:, 1]


Mejores hiperparámetros: {'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'exponential', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Gradient Boosting - Modelo optimizado:
Accuracy Train: 0.9851543942992874
Accuracy Train: 0.9192399049881235
ROC AUC: 0.9551537515375152
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      0.92      0.89       150
           1       0.95      0.92      0.94       271

    accuracy                           0.92       421
   macro avg       0.91      0.92      0.91       421
weighted avg       0.92      0.92      0.92       421



## Evaluacion

In [ ]:

# Evaluacion del modelo
accuracy_test_gb_op = accuracy_score(y_test, y_pred_gb_op)
roc_auc_gb_op = roc_auc_score(y_test, y_prob_gb_op)
precision_gb_op = precision_score(y_test, y_pred_gb_op)
recall_gb_op = recall_score(y_test, y_pred_gb_op)
f1_gb_op = f1_score(y_test, y_pred_gb_op)

classification_report_gb_op = classification_report(y_test, y_pred_gb_op)


# Metricas
print("Gradient boosting - Modelo optimizado")
print("Accuracy", accuracy_test_gb_op)
print("Precision:", precision_gb_op)
print("Roc-Auc:", roc_auc_gb_op)
print("Recall:", recall_gb_op)
print("F1 Score:", f1_gb_op)

print("Reporte de clasificación:\n", classification_report_gb_op)

Mejores hiperparámetros: {'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'exponential', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Gradient Boosting - Modelo optimizado:
Accuracy Train: 0.9851543942992874
Accuracy Train: 0.9192399049881235
ROC AUC: 0.9551537515375152
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      0.92      0.89       150
           1       0.95      0.92      0.94       271

    accuracy                           0.92       421
   macro avg       0.91      0.92      0.91       421
weighted avg       0.92      0.92      0.92       421

## Comparacion modelos

In [ ]:

estimators = {"Gradient boosting basico": gb, "Gradient boosting optimizado": gb_op}
metrics = {"Accuracy": "accuracy", "Precision":"precision", "Roc-Auc":"roc_auc", "Recall":"recall","F1":"f1"}

print("Evaluacion")
evaluate(estimators, metrics, X_test, y_test)


print("Evaluacion Test final")
evaluate(estimators, metrics, X_test_final, y_test_final)

# Histogram Gradient Boosting

## Modelo basico

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
hgb = HistGradientBoostingClassifier(
    max_iter=100,           # Número de iteraciones (equivalente a n_estimators)
    learning_rate=0.1,      # Tasa de aprendizaje
    max_depth=5,            # Profundidad máxima de los árboles
    random_state=random_state        # Semilla para reproducibilidad
)

# Entrenar el modelo
hgb.fit(X_train, y_train)

# Predicciones
y_pred_hgb = hgb.predict(X_test)
y_prob_hgb = hgb.predict_proba(X_test)[:, 1]


# Evaluacion del modelo
accuracy_train_hgb = .score(X_train, y_train)
accuracy_test_hgb = accuracy_score(y_test, y_pred_hgb)
roc_auc_hgb = roc_auc_score(y_test, y_prob_hgb)
precision_hgb = precision_score(y_test, y_pred_hgb)
recall_hgb = recall_score(y_test, y_pred_hgb)
f1_hgb = f1_score(y_test, y_pred_hgb)
classification_report_gb = classification_report(y_test, y_pred_hgb)


# Metricas
print("Histogram Gradient Boosting - Modelo básico")
print("Accuracy Train", accuracy_train_hgb)
print("Accuracy Test", accuracy_test_hgb)
print("Precision:", precision_hgb)
print("Roc-Auc:", roc_auc_hgb)
print("Recall:", recall_hgb)
print("F1 Score:", f1_hgb)

print("Reporte de clasificación:\n", classification_report_hgb)

Histogram Gradient Boosting - Modelo básico:
Accuracy Train 0.995249406175772
Accuracy Test 0.9216152019002375
ROC AUC: 0.9589175891758918
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      0.93      0.89       150
           1       0.96      0.92      0.94       271

    accuracy                           0.92       421
   macro avg       0.91      0.92      0.92       421
weighted avg       0.92      0.92      0.92       421



## Optimización

In [ ]:
# hiperparámetros
grid_hgb = {
    'max_iter': [50, 100, 200],            # Número de iteraciones
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],    # Tasa de aprendizaje
    'max_leaf_nodes': [15, 31, 63, 127],
    'max_depth': [3, 5, 7],               # Profundidad máxima
    'l2_regularization': [0, 0.1, 1.0],   # Regularización L2
    'min_samples_leaf': [20, 50, 100]     # Muestras mínimas por hoja
}

hgb_op = optimize(
    estimator=hgb,  # Modelo base
    grid=grid_hgb,  # Hiperparámetros a probar
    X=X_train,  # Conjunto de características de entrenamiento
    y=y_train,  # Etiquetas de entrenamiento
    cv=5,  # Validación cruzada con 5 pliegues
    scoring='accuracy',
    refit='accuracy'
)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", gb_op.best_params_)

# Entrenamiento
grid_search_hist_gb.fit(X_train, y_train)

# Predicciones
y_pred_hgb_op = hist_gb_op.predict(X_test)
y_prob_hgb_op = hist_gb_op.predict_proba(X_test)[:, 1]


Mejores hiperparámetros: {'l2_regularization': 1.0, 'learning_rate': 0.2, 'loss': 'log_loss', 'max_depth': 3, 'max_iter': 50, 'max_leaf_nodes': 15, 'min_samples_leaf': 20}
Histogram Gradient Boosting - Modelo optimizado:
Accuracy Train 0.9685273159144893
Accuracy Test 0.9216152019002375
ROC AUC: 0.9623370233702337
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      0.93      0.89       150
           1       0.96      0.92      0.94       271

    accuracy                           0.92       421
   macro avg       0.91      0.92      0.92       421
weighted avg       0.92      0.92      0.92       421



## Evaluacion

In [ ]:

# Evaluacion del modelo
accuracy_test_hgb_op = accuracy_score(y_test, y_pred_hgb_op)
roc_auc_hgb_op = roc_auc_score(y_test, y_prob_hgb_op)
precision_hgb_op = precision_score(y_test, y_pred_hgb_op)
recall_hgb_op = recall_score(y_test, y_pred_hgb_op)
f1_hgb_op = f1_score(y_test, y_pred_hgb_op)

classification_report_hgb_op = classification_report(y_test, y_pred_hgb_op)

# Metricas
print("Histogram Gradient boosting - Modelo optimizado")
print("Accuracy", accuracy_test_hgb_op)
print("Precision:", precision_hgb_op)
print("Roc-Auc:", roc_auc_hgb_op)
print("Recall:", recall_hgb_op)
print("F1 Score:", f1_hgb_op)

print("Reporte de clasificación:\n", classification_report_hgb_op)

## Comparación modelos

In [ ]:
estimators = {"Histogram gradient boosting basico": hgb, "Histogram gradient boosting optimizado": hgb_op}
metrics = {"Accuracy": "accuracy", "Precision":"precision", "Roc-Auc":"roc_auc", "Recall":"recall","F1":"f1"}

print("Evaluacion")
evaluate(estimators, metrics, X_test, y_test)


print("Evaluacion Test final")
evaluate(estimators, metrics, X_test_final, y_test_final)

# Evaluacion de modelos